In [1]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
pima = pd.read_csv("./cl2.csv")


In [2]:
# Split the data into features (X) and target variable (y)
X = pima.drop(columns='target')
y = pima['target']

In [3]:
# Number of negative and positive cases in the data
num_obs = len(pima)
negative = len(pima.loc[pima['target'] == 0])
positive = len(pima.loc[pima['target'] == 1])
print("Number of negative cases:  {0} ({1:2.2f}%)".format(negative, ((1.00 * negative)/(1.0 * num_obs)) * 100))
print("Number of positve cases:  {0} ({1:2.2f}%)".format(positive, ((1.00 * positive)/(1.0 * num_obs)) * 100))

Number of negative cases:  373 (48.50%)
Number of positve cases:  396 (51.50%)


In [4]:
# Split xscale
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42) 

In [5]:
print(X_train.shape,X_test.shape)

(615, 14) (154, 14)


In [6]:
from xgboost import XGBClassifier 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
xgb = XGBClassifier(objective = 'binary:logistic')

xgb.fit(X_train, y_train)
print()
y_pred = xgb.predict(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred))


Confusion Matrix: 
 [[79  2]
 [ 0 73]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        81
           1       0.97      1.00      0.99        73

    accuracy                           0.99       154
   macro avg       0.99      0.99      0.99       154
weighted avg       0.99      0.99      0.99       154



In [7]:
import argparse
import warnings
from typing import Union
from logging import INFO
from datasets import Dataset, DatasetDict
import xgboost as xgb

import flwr as fl
from flwr_datasets import FederatedDataset
from flwr.common.logger import log
from flwr.common import (
    Code,
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    GetParametersIns,
    GetParametersRes,
    Parameters,
    Status,
)
from flwr_datasets.partitioner import IidPartitioner


warnings.filterwarnings("ignore", category=UserWarning)

# def transform_dataset_to_dmatrix(data: Union[Dataset, DatasetDict]) -> xgb.core.DMatrix:
#     """Transform dataset to DMatrix format for xgboost."""
#     x = data["inputs"]
#     y = data["label"]
#     new_data = xgb.DMatrix(x, label=y)
#     return new_data

# # Train/test splitting
# train_data, valid_data, num_train, num_val = X_train, X_test, y_train, y_test
num_train = 420
num_val = 105


xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# Reformat data to DMatrix for xgboost
log(INFO, "Reformatting data...")
# train_dmatrix = transform_dataset_to_dmatrix(train_data)
# valid_dmatrix = transform_dataset_to_dmatrix(valid_data)
train_dmatrix = xgb_train
valid_dmatrix = xgb_test

import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report


# Hyper-parameters for xgboost training
num_local_round = 5
params = {
    "objective": "binary:logistic",
    "eta": 0.5,  # Learning rate
    "max_depth": 25,
    "eval_metric": "error",
    "nthread": 16,
    "num_parallel_tree": 1,
    "subsample": 1,
    "tree_method": "hist",
}


# Define Flower client
class XgbClient(fl.client.Client):
    def __init__(self):
        self.bst = None
        self.config = None

    def get_parameters(self, ins: GetParametersIns) -> GetParametersRes:
        _ = (self, ins)
        return GetParametersRes(
            status=Status(
                code=Code.OK,
                message="OK",
            ),
            parameters=Parameters(tensor_type="", tensors=[]),
        )

    def _local_boost(self):
        # Update trees based on local training data.
        for i in range(num_local_round):
            self.bst.update(train_dmatrix, self.bst.num_boosted_rounds())

        # Extract the last N=num_local_round trees for sever aggregation
        bst = self.bst[
            self.bst.num_boosted_rounds()
            - num_local_round : self.bst.num_boosted_rounds()
        ]

        return bst

    def fit(self, ins: FitIns) -> FitRes:
        if not self.bst:
            # First round local training
            log(INFO, "Start training at round 1")
            bst = xgb.train(
                params,
                train_dmatrix,
                num_boost_round=num_local_round,
                evals=[(valid_dmatrix, "validate"), (train_dmatrix, "train")],
            )
            self.config = bst.save_config()
            self.bst = bst
        else:
            for item in ins.parameters.tensors:
                global_model = bytearray(item)

            # Load global model into booster
            self.bst.load_model(global_model)
            self.bst.load_config(self.config)

            bst = self._local_boost()

        local_model = bst.save_raw("json")
        local_model_bytes = bytes(local_model)

        return FitRes(
            status=Status(
                code=Code.OK,
                message="OK",
            ),
            parameters=Parameters(tensor_type="", tensors=[local_model_bytes]),
            num_examples=num_train,
            metrics={},
        )

    def evaluate(self, ins: EvaluateIns) -> EvaluateRes:
        eval_results = self.bst.eval_set(
            evals=[(valid_dmatrix, "valid")],
            iteration=self.bst.num_boosted_rounds() - 1,
        )
        auc = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
        
        preds = self.bst.predict(valid_dmatrix)
        y_pred = np.multiply(preds,100)
        y_pred = y_pred.astype(int)
        a = [1 if i >= 50 else 0 for i in y_pred]
        print("Confusion Matrix: \n", confusion_matrix(y_test, a))
        print(metrics.classification_report(y_test,a))
        

        return EvaluateRes(
            status=Status(
                code=Code.OK,
                message="OK",
            ),
            loss=0.0,
            num_examples=num_val,
            metrics={"error": auc},
        )


# Start Flower client
fl.client.start_client(server_address="127.0.0.1:8080", client=XgbClient().to_client())

c:\Users\fafaf\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO flwr 2024-05-12 17:56:44,209 | 141506961.py:44 | Reformatting data...
INFO flwr 2024-05-12 17:56:44,232 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-05-12 17:56:44,234 | connection.py:55 | ChannelConnectivity.IDLE
DEBUG flwr 2024-05-12 17:56:44,237 | connection.py:55 | ChannelConnectivity.READY
INFO flwr 2024-05-12 17:56:48,060 | 141506961.py:101 | Start training at round 1


[0]	validate-error:0.14935	train-error:0.04878
[1]	validate-error:0.09740	train-error:0.02602
[2]	validate-error:0.03247	train-error:0.00813
[3]	validate-error:0.03247	train-error:0.00813
[4]	validate-error:0.03247	train-error:0.00488
Confusion Matrix: 
 [[79  2]
 [ 3 70]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        81
           1       0.97      0.96      0.97        73

    accuracy                           0.97       154
   macro avg       0.97      0.97      0.97       154
weighted avg       0.97      0.97      0.97       154

Confusion Matrix: 
 [[80  1]
 [ 1 72]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99        81
           1       0.99      0.99      0.99        73

    accuracy                           0.99       154
   macro avg       0.99      0.99      0.99       154
weighted avg       0.99      0.99      0.99       154

Confusion Matrix: 
 [[80  1]
 [ 1

DEBUG flwr 2024-05-12 17:56:48,723 | connection.py:220 | gRPC channel closed


Confusion Matrix: 
 [[81  0]
 [ 0 73]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        81
           1       1.00      1.00      1.00        73

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154

Confusion Matrix: 
 [[81  0]
 [ 0 73]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        81
           1       1.00      1.00      1.00        73

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154

Confusion Matrix: 
 [[81  0]
 [ 0 73]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        81
           1       1.00      1.00      1.00        73

    accuracy                           1.00       154
   macro av

INFO flwr 2024-05-12 17:56:48,724 | app.py:398 | Disconnect and shut down
